In [1]:
import sys
sys.path.insert(0,'..')

from sktime_analysis import *

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sktime.classification.compose import (
    ColumnEnsembleClassifier
)
from sktime.classification.frequency_based import RandomIntervalSpectralForest
from sktime.transformers.panel.compose import ColumnConcatenator

In [2]:
n_samples = 3
X, y = create_data_for_sktime(n_samples, spring_constants=False)

n_experiments = 10

#time series concatenation
steps = [
    ("concatenate", ColumnConcatenator()),
    ("classify", RandomIntervalSpectralForest(n_estimators=100)),
]
clf_concat = Pipeline(steps)
#column ensembling
est = [
        ("RISE0", RandomIntervalSpectralForest(n_estimators=100), [0]),
        ("RISE1", RandomIntervalSpectralForest(n_estimators=100), [1]),
    ]
clf_ens = ColumnEnsembleClassifier(estimators=est)
    

test_size = .5
scores = {'concat':np.empty(n_experiments), 'ens':np.empty(n_experiments)}
for i in range(n_experiments):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    clf_concat.fit(X_train, y_train)
    scores['concat'][i] = clf_concat.score(X_test, y_test)
    clf_ens.fit(X_train, y_train)
    scores['ens'][i] = clf_ens.score(X_test, y_test)
    

    
print('Test Size:', test_size)
print('Column Concatenation Score:', np.mean(scores['concat']))

print('Column Ensemble Score:', np.mean(scores['ens']))

Test Size: 0.5
Column Concatenation Score: 1.0
Column Ensemble Score: 0.9925925925925926
